<a href="https://colab.research.google.com/github/niteshkamboj2811/Python_Practice/blob/main/Strategy_%26_Analytics_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Preparing environment


In [4]:
#upgrading SQLite
"""This step is necessary as default sqlite3 is version 3.22 in google colab which has a limitations to running window funtions"""
#Stackoverflow article: https://stackoverflow.com/questions/62735105/sqlite-window-function-not-running-in-google-colab-notebook

!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/

Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /content/_sqlite3.cpython-37m-x86_64-linux-gnu.so
6.23MB [00:00, 97.4MB/s]


In [ ]:
# MENU: Runtime > Restart runtime
#OR
import os
os._exit(00)

In [7]:
pip install csv-to-sqlite #To load csv into sqlite3

In [8]:
import csv_to_sqlite
import pandas as pd
import sqlite3
sqlite3.sqlite_version  # '3.34.1'

'3.34.1'

#Loading subscriptions.csv and visits.csv into a db

In [1]:
#Please upload the "subscriptions.csv"
from google.colab import files
subscriptions_csv = files.upload()

Saving subscriptions.csv to subscriptions.csv


In [2]:
#Please upload the "visits.csv"
from google.colab import files
visits_csv = files.upload()

Saving visits.csv to visits.csv


In [9]:
#Writing "subscriptions.csv" and "visits.csv" into tables
options = csv_to_sqlite.CsvOptions(typing_style="full", encoding="windows-1250") 
input_files = ["subscriptions.csv", "visits.csv"]
csv_to_sqlite.write_csv(input_files, "output.sqlite", options)

Written 61316 rows into 2 tables in 0.507 seconds


61316

In [10]:
#Creating engine
con=sqlite3.connect('output.sqlite')

#Exploring the data

In [11]:
pd.read_sql_query("""SELECT *
  FROM subscriptions
 LIMIT 10;""",con)

,subscription_id,account_id,subscription_start_date,subscription_end_date,trial_start_date,trial_end_date,revenue
0,96d87f27-c4e3-401e-9efd-795834b12e1b,914fddea-7a72-4712-82dd-5b4590f9b670,2019-01-08,2020-01-08,2019-01-01,2019-01-08,1016.28
1,91838d2f-5cc1-448c-95d2-05dce13f0090,7e6ccf6b-16e1-4ea1-9057-ead81ac1cedd,2019-01-01,2020-01-01,,,755.67
2,7cb08fd7-9440-4a17-9cbd-674954d9b1c5,f7b65b83-2e33-4926-b36a-f37fead86623,2019-01-01,2020-01-01,,,999.89
3,ab2b038d-e56c-4102-8662-dedbff889235,f9b33d91-66d1-485c-8c6e-d27bdb46d605,2019-01-08,2020-01-08,2019-01-01,2019-01-08,971.82
4,183c7790-06fb-40f8-acf4-f3c78db265fd,c5124861-4163-409d-a419-c87681537371,2019-01-02,2020-01-02,,,1280.93
5,0adab5fa-ec35-4e1e-9da5-70bd45d684f6,c0463e2b-ca34-4f2c-b50f-a418df8923de,,,2019-01-02,2019-01-09,
6,73f54f57-d366-43e0-8408-785332db207b,70840455-af47-4816-8792-6c7b8d18fb23,2019-01-02,2020-01-02,,,905.53
7,ece4a01a-9a96-4bee-9006-577405adc05e,f69dca85-131e-45a5-802a-8c197f0b1137,,,2019-01-02,2019-01-09,
8,6549855e-bc71-4f3f-abfc-2f6cd626e0b7,b25c453e-04e7-45f3-9279-289c1ff2e805,2019-01-02,2020-01-02,,,1198.34
9,21da15c6-bd98-44ad-9e55-ba43c7e640ae,eb185350-ba04-4913-b78c-7f0672cdf34f,2019-01-10,2020-01-10,2019-01-03,2019-01-10,1198.38


In [12]:
pd.read_sql_query("""SELECT * 
  FROM visits
 LIMIT 10;""",con)

,visit_id,account_id,day,region,language,channel,landing_page
0,4b987205-5da4-403c-b7d4-b952db264794,914fddea-7a72-4712-82dd-5b4590f9b670,2019-01-01,US/Canada,English,video,A
1,93d005cd-2990-4432-ae98-bd11c9c28f63,7e6ccf6b-16e1-4ea1-9057-ead81ac1cedd,2019-01-01,US/Canada,English,organic search,G
2,0ec30176-7401-480c-8630-92da7e020a54,,2019-01-01,US/Canada,English,organic search,E
3,4374d20b-d6bc-4b76-9fc8-a07847af79f9,,2019-01-01,US/Canada,English,organic search,G
4,f64662d1-ce3f-427a-b175-55d54c123c72,,2019-01-01,US/Canada,Spanish,organic search,E
5,eb594d0b-80ec-44fe-97a5-85afac3931a9,,2019-01-01,US/Canada,English,organic search,D
6,d1dc5275-90f9-4578-810d-73791925482e,,2019-01-01,US/Canada,English,organic search,J
7,d2bd6d7a-0f61-4323-8bbc-1e934a7368fd,f7b65b83-2e33-4926-b36a-f37fead86623,2019-01-01,US/Canada,English,organic search,H
8,83674a56-2bc3-47ed-95ee-598ee782d333,f9b33d91-66d1-485c-8c6e-d27bdb46d605,2019-01-01,US/Canada,English,paid search,E
9,8e18cb8d-62ca-486c-aafc-df68f886033a,,2019-01-01,US/Canada,English,video,A


In [13]:
#Exploring the data for null values with Pandas
subscriptions_df = pd.read_csv('subscriptions.csv')
visits_df = pd.read_csv('visits.csv')

print(subscriptions_df.isnull().any())
print('\n')
print(visits_df.isnull().any())

subscription_id            False
account_id                 False
subscription_start_date     True
subscription_end_date       True
trial_start_date            True
trial_end_date              True
revenue                     True
dtype: bool


visit_id        False
account_id       True
day             False
region          False
language        False
channel         False
landing_page    False
dtype: bool


####Due to the limitation of UPDATE SET functionality in pd.read_sql_query we will Null the empty record on the fly.

---



#Basic Question

##Question 1: How much revenue has each region generated over the past two years?

In [14]:
Answer_1 = pd.read_sql_query("""SELECT region,
       ROUND(SUM(revenue), 0) AS Revenue
  FROM visits AS v,
       subscriptions AS s
 WHERE v.account_id = s.account_id
 GROUP BY 1
 ORDER BY 2 DESC;""", con)

Answer_1

,region,Revenue
0,Latin America,2373439.0
1,Asia,1977364.0
2,US/Canada,1583260.0
3,"Europe, Middle East, & Africa (Non-English)",1317910.0
4,Other English-Speaking Countries,1135564.0


##Question 2: Which channel has the highest revenue per visitor?

In [15]:
Answer_2 = pd.read_sql_query("""SELECT channel,
       ROUND(SUM(revenue) / COUNT(DISTINCT visit_id), 0) AS 'Revenue_Per_Visitor($)'
  FROM visits AS v,
       subscriptions AS s
 WHERE v.account_id = s.account_id
 GROUP BY 1
 ORDER BY 2 DESC;""", con)

Answer_2

,channel,Revenue_Per_Visitor($)
0,display,917.0
1,partner referral,761.0
2,email,686.0
3,organic search,579.0
4,video,566.0
5,social,519.0
6,paid search,500.0


In [16]:
print(f"ANSWER: Highest revenue is generateg by '{Answer_2.iloc[0][0]}' Channel with 'Revenue_Per_Visitor' of ${Answer_2.iloc[0][1]}")

ANSWER: Highest revenue is generateg by 'display' Channel with 'Revenue_Per_Visitor' of $917.0


##Question 3: Which landing page has the highest overall conversion rate (% of visitors that become subscribers)?

###First, let's calculate the conversion rate

In [20]:
CR = pd.read_sql_query("""SELECT (Subscribers * 100 / Visitor) AS 'Overall_Conversion_Rate(%)'
  FROM (
           SELECT COUNT(CASE WHEN revenue != '' THEN s.account_id ELSE NULL END) AS Subscribers,
                  COUNT(DISTINCT visit_id) AS Visitor
             FROM visits AS v
                  LEFT JOIN
                  subscriptions AS s ON v.account_id = s.account_id
       );""", con)

CR

,Overall_Conversion_Rate(%)
0,17


###Now, calculate which landing page has highest conversion rate

In [21]:
landingPageHighCR = pd.read_sql_query("""SELECT *
  FROM (
           SELECT landing_page,
                  (Subscribers * 100 / Visitor) AS 'Conversion_Rate(%)',
                  ROW_NUMBER() OVER (ORDER BY Subscribers * 100 / Visitor DESC) AS Rank
             FROM (
                      SELECT landing_page,
                             COUNT(CASE WHEN revenue !='' THEN s.account_id ELSE NULL END) AS Subscribers,
                             COUNT(DISTINCT visit_id) AS visitor
                        FROM visits AS v
                             LEFT JOIN
                             subscriptions AS s ON v.account_id = s.account_id
                       GROUP BY 1
                  )
       )
 WHERE Rank = 1;""", con)

landingPageHighCR

,landing_page,Conversion_Rate(%),Rank
0,D,21,1


In [22]:
print(f"ANSWER: Landing Page '{landingPageHighCR.iloc[0][0]}' has the highest conversion rate of '{landingPageHighCR.iloc[0][1]}%'")

ANSWER: Landing Page 'D' has the highest conversion rate of '21%'


###Now checking if the above result of Landing Page: "D" with highest conversion rate stands true against Regions

In [23]:
landingPageHighCR_region = pd.read_sql_query("""SELECT *
  FROM (
           SELECT landing_page,
                  region,
                  (Subscribers * 100 / visitor) AS 'Conversion_Rate(%)',
                  ROW_NUMBER() OVER (PARTITION BY region ORDER BY Subscribers * 100 / Visitor DESC) AS Rank
             FROM (
                      SELECT landing_page,
                             region,
                             COUNT(CASE WHEN revenue != '' THEN s.account_id ELSE NULL END) AS Subscribers,
                             COUNT(DISTINCT visit_id) AS visitor
                        FROM visits AS v
                             LEFT JOIN
                             subscriptions AS s ON v.account_id = s.account_id
                       GROUP BY 1,
                                2
                  )
       )
 WHERE Rank = 1
 ORDER BY 3 DESC;""", con)

landingPageHighCR_region

,landing_page,region,Conversion_Rate(%),Rank
0,D,Other English-Speaking Countries,23,1
1,D,Latin America,22,1
2,D,"Europe, Middle East, & Africa (Non-English)",21,1
3,D,Asia,20,1
4,D,US/Canada,19,1


In [24]:
print(f"ANSWER: Landing Page '{landingPageHighCR.iloc[0][0]}' certainly shows the same conversion rate through all regions.")

ANSWER: Landing Page 'D' certainly shows the same conversion rate through all regions.


###Now checking if the above result of Landing Page: "D" with highest conversion rate stands true against Languages

In [25]:
landingPageHighCR_language = pd.read_sql_query("""SELECT *
  FROM (
           SELECT landing_page,
                  language,
                  (Subscribers * 100 / visitor) AS 'Conversion_Rate(%)',
                  ROW_NUMBER() OVER (PARTITION BY language ORDER BY Subscribers * 100 / Visitor DESC) AS Rank
             FROM (
                      SELECT landing_page,
                             language,
                             COUNT(CASE WHEN revenue != '' THEN s.account_id ELSE NULL END) AS Subscribers,
                             COUNT(DISTINCT visit_id) AS Visitor
                        FROM visits AS v
                             LEFT JOIN
                             subscriptions AS s ON v.account_id = s.account_id
                       GROUP BY 1,
                                2
                  )
       )
 WHERE Rank = 1
 ORDER BY 3 DESC;""", con)

landingPageHighCR_language

,landing_page,language,Conversion_Rate(%),Rank
0,I,Portuguese,23,1
1,D,Spanish,23,1
2,G,Other,21,1
3,D,English,20,1
4,D,Other_Romance,20,1


In [26]:
print("ANSWER: Landing Page D still seems to have highest conversion rate for all languages.")

ANSWER: Landing Page D still seems to have highest conversion rate for all languages.


###Now checking if the above result of Landing Page: "D" with highest conversion rate stands true against Channels

In [27]:
landingPageHighCR_channel = pd.read_sql_query("""SELECT *
  FROM (
           SELECT landing_page,
                  channel,
                  (Subscribers * 100 / Visitor) AS 'Conversion_Rate(%)',
                  ROW_NUMBER() OVER (PARTITION BY channel ORDER BY Subscribers * 100 / Visitor DESC) AS Rank
             FROM (
                      SELECT landing_page,
                             channel,
                             COUNT(CASE WHEN revenue != '' THEN s.account_id ELSE NULL END) AS Subscribers,
                             COUNT(DISTINCT visit_id) AS Visitor
                        FROM visits AS v
                             LEFT JOIN
                             subscriptions AS s ON v.account_id = s.account_id
                       GROUP BY 1,
                                2
                  )
       )
 WHERE Rank = 1
 ORDER BY 3 DESC;""", con)

landingPageHighCR_channel

,landing_page,channel,Conversion_Rate(%),Rank
0,I,email,34,1
1,D,display,27,1
2,D,video,24,1
3,D,partner referral,20,1
4,D,paid search,19,1
5,A,organic search,18,1
6,D,social,15,1


In [28]:
print("ANSWER: Landing Page D still seems to have high conversion rate for all channels except email and organic search. In all it still holds true for channels too")

ANSWER: Landing Page D still seems to have high conversion rate for all channels except email and organic search. In all it still holds true for channels too


###Landing page "D" has a highest conversion rate genreally and for regions and languages. For channels, it still is **second highest**. We can conclude Landing Page D does see a highest conversion rate of visitors into subscribers.

---



#Strategic Questions

##Options for increasing revenue by 20% in US/Canada region

###Option 1: Increasing reveue by attracting more number of visitors.

> Assumption: The subscription rate remains constant.



In [29]:
#Calculating the number of visitors and total revenue
PresentState = pd.read_sql_query("""SELECT COUNT(DISTINCT visit_id) AS Present_Visitors_Count,
       ROUND(SUM(revenue), 0) AS 'Present_Revenue($)'
  FROM visits AS v,
       subscriptions AS s
 WHERE v.account_id = s.account_id AND 
       region LIKE 'US%';
""", con)

PresentState

,Present_Visitors_Count,Present_Revenue($)
0,3036,1583260.0


In [30]:
#Calculating the number of visitors we need to increase to incease the revenue by 20%
TargetState = pd.read_sql_query("""SELECT ROUND(Present_Visitors * Target_Revenue / Present_Revenue) AS Target_Visitor_Count,
       Target_Revenue AS 'Target_Revenue($)'
  FROM (
           SELECT COUNT(DISTINCT visit_id) AS Present_Visitors,
                  ROUND(SUM(revenue) * 1.2, 0) AS Target_Revenue,
                  ROUND(SUM(revenue), 0) AS Present_Revenue
             FROM visits AS v,
                  subscriptions AS s
            WHERE v.account_id = s.account_id AND 
                  region LIKE 'US%'
       );""", con)

TargetState

,Target_Visitor_Count,Target_Revenue($)
0,3643.0,1899912.0


In [31]:
#So, we need to attract more visitors to gain revenue by 20% with same conversion rate.
print(f"RECOMMENDATION: We need attract ~{TargetState['Target_Visitor_Count'][0] - PresentState['Present_Visitors_Count'][0]} more visitors to increase revenue by 20%.")

RECOMMENDATION: We need attract ~607.0 more visitors to increase revenue by 20%.


In [32]:
print("RECOMMENDATION: The other option is to re-work the traffic management and push the visitors to land on Landing Page D as it clearly has better conversion rates as seen in Question 3")

RECOMMENDATION: The other option is to re-work the traffic management and push the visitors to land on Landing Page D as it clearly has better conversion rates as seen in Question 3


###Option 2: Increasing reveue by increasing the duration of trial period.

> Assumption: The subscription rate remains constant.


In [34]:
#Present length of trail period
PresentTrialState = pd.read_sql_query("""SELECT DISTINCT CAST (JulianDay(trial_end_date) - JulianDay(trial_start_date) AS INTEGER) AS Current_Trial_Period,
                ROUND(SUM(revenue), 0) AS 'Present_Revenue($)'
  FROM visits AS v,
       subscriptions AS s
 WHERE v.account_id = s.account_id AND 
       region LIKE 'US%';
""", con)

PresentTrialState

,Current_Trial_Period,Present_Revenue($)
0,7,1583260.0


In [35]:
#Calculating the increased trial period length to incease the revenue by 20%
TargetTrialState = pd.read_sql_query("""SELECT ROUND( (Current_Trial_Period * Target_Revenue / Present_Revenue), 1) AS Target_Trial_Period,
       Target_Revenue AS 'Target_Revenue($)'
  FROM (
           SELECT DISTINCT CAST (JulianDay(trial_end_date) - JulianDay(trial_start_date) AS INTEGER) AS Current_Trial_Period,
                           ROUND(SUM(revenue), 0) AS Present_Revenue,
                           ROUND(SUM(revenue) * 1.2, 0) AS Target_Revenue
             FROM visits AS v,
                  subscriptions AS s
            WHERE v.account_id = s.account_id AND 
                  region LIKE 'US%'
       );""", con)

TargetTrialState

,Target_Trial_Period,Target_Revenue($)
0,8.4,1899912.0


In [36]:
#So, we need to increase the trial period duration to gain revenue by 20% with same subscription rate.
print(f"RECOMMENDATION: We should increase the trial duration from {PresentTrialState['Current_Trial_Period'][0]} to {TargetTrialState['Target_Trial_Period'][0]} days to increase revenue by 20% considering the conversion rate remains unchanged.")

RECOMMENDATION: We should increase the trial duration from 7 to 8.4 days to increase revenue by 20% considering the conversion rate remains unchanged.


###Option 3: Increasing the conversion rate of people who try but do not subscribe.

####Fact: The revenue is generated by visitor subscribing to product(s).

> We will first look at the ratio between visitors and customers. We are using term customers for the poeple who have tried the product or subscribed or both. This is same as description for the account_id in visits table.



In [37]:
CurrentTrialSubState = pd.read_sql_query("""SELECT COUNT(CASE WHEN v.account_id != '' THEN v.account_id ELSE NULL END) * 100 / COUNT(DISTINCT visit_id) AS Current_Customers_Percentage,
       COUNT(CASE WHEN revenue != '' THEN s.account_id ELSE NULL END) * 100 / COUNT(DISTINCT visit_id) AS Current_Subscribers_Percentage,
       ROUND(SUM(revenue), 0) AS 'Current_Revenue($)'
  FROM visits AS v
       LEFT JOIN
       subscriptions AS s ON v.account_id = s.account_id
 WHERE region LIKE 'US%';
""", con)

CurrentTrialSubState

,Current_Customers_Percentage,Current_Subscribers_Percentage,Current_Revenue($)
0,27,15,1583260.0


In [38]:
#Calculating the difference between Current_Customers_Percentage and Current_Subscribers_Percentage; This will give us percent of visitors who tried the product but did not subscribe to it.
print(f"INSIGHT: Percent of visitors who tried the product but did not subscribe (US/Canada): {CurrentTrialSubState['Current_Customers_Percentage'][0] - CurrentTrialSubState['Current_Subscribers_Percentage'][0]} %")

INSIGHT: Percent of visitors who tried the product but did not subscribe (US/Canada): 12 %



> Since the trial do not generate any revenue, only 15% of visitor are actually paying for the product.


> So, if only 15% subscribers make up for the revenue of 1583260. Let's calculate what percentage of subscribers do we need to reach our goal of $1899912 (1.20*Present Revenue)




In [39]:
TargetTrialSubState = pd.read_sql_query("""SELECT Target_Revenue * Current_Customers_Percentage / Present_Revenue AS Target_Customers_Percentage,
       Target_Revenue * Current_Subscribers_Percentage / Present_Revenue AS Target_Subscribers_Percentage,
       Target_Revenue AS 'Target_Revenue($)'
  FROM (
           SELECT COUNT(CASE WHEN v.account_id != '' THEN v.account_id ELSE NULL END) * 100 / COUNT(DISTINCT visit_id) AS Current_Customers_Percentage,
                  COUNT(CASE WHEN revenue != '' THEN s.account_id ELSE NULL END) * 100 / COUNT(DISTINCT visit_id) AS Current_Subscribers_Percentage,
                  ROUND(SUM(revenue), 0) AS Present_Revenue,
                  ROUND(SUM(revenue) * 1.2, 0) AS Target_Revenue
             FROM visits AS v
                  LEFT JOIN
                  subscriptions AS s ON v.account_id = s.account_id
            WHERE region LIKE 'US%'
       );""", con)

TargetTrialSubState

,Target_Customers_Percentage,Target_Subscribers_Percentage,Target_Revenue($)
0,32.4,18.0,1899912.0


In [40]:
#So, we need to increase the trial period duration to gain revenue by 20% with same subscription rate.
print(f"RECOMMENDATION: We should taget to increase the Customers_Percentage (Visitor who try and/or subscribe) from {CurrentTrialSubState['Current_Customers_Percentage'][0]} to {TargetTrialSubState['Target_Customers_Percentage'][0]} and target to grow Subscribers (Visitors who subscribe) conversion rate from {CurrentTrialSubState['Current_Subscribers_Percentage'][0]}% to {TargetTrialSubState['Target_Subscribers_Percentage'][0]}% to increase revenue by 20%")

RECOMMENDATION: We should taget to increase the Customers_Percentage (Visitor who try and/or subscribe) from 27 to 32.4 and target to grow Subscribers (Visitors who subscribe) conversion rate from 15% to 18.0% to increase revenue by 20%


###Option 4: Re-budgeting the marketing expense by evaluating **Channels** performance.

> We will look at a factor which is directly related to revenue - Channel and see how we can optimize our makreting expenditure with it.

> We will try to flag the least performing channels so that we can use that expenditure towards most performing channels.



In [41]:
ChannelPerformance = pd.read_sql_query("""WITH channel_rev AS (
    SELECT channel,
           ROUND(sum(revenue) ) AS Present_Revenue,
           ROUND(SUM(revenue) * 1.2, 0) AS Target_Revenue,
           DENSE_RANK() OVER (ORDER BY SUM(revenue) DESC) AS Rank
      FROM visits AS v,
           subscriptions AS s
     WHERE v.account_id = s.account_id AND 
           region LIKE 'US%'
     GROUP BY 1
)
SELECT channel,
       Present_Revenue,
       Target_Revenue AS 'Target_Revenue($)',
       Rank
  FROM channel_rev;""", con)

ChannelPerformance

,channel,Present_Revenue,Target_Revenue($),Rank
0,organic search,702308.0,842769.0,1
1,paid search,369825.0,443790.0,2
2,partner referral,258681.0,310417.0,3
3,display,184597.0,221516.0,4
4,video,65533.0,78640.0,5
5,social,1739.0,2087.0,6
6,email,578.0,694.0,7


In [42]:
print(f"INSIGHT: '{ChannelPerformance.iloc[1][0]}' channel contributed to 2 years revenue by {(ChannelPerformance['Present_Revenue'][1]*100/ChannelPerformance['Present_Revenue'].sum()).round()}%")
print('\n')
print(f"RECOMMENDATION: We hardly have any control over 'Organic Searches' but we definitely have leverage over 'Paid Searches'. We should re-budget to spend more on 'Paid Searches' and less on 'Email' or 'Social' channels.")

INSIGHT: 'paid search' channel contributed to 2 years revenue by 23.0%


RECOMMENDATION: We hardly have any control over 'Organic Searches' but we definitely have leverage over 'Paid Searches'. We should re-budget to spend more on 'Paid Searches' and less on 'Email' or 'Social' channels.


###Option 5: Re-budgeting the marketing expense by evaluating **seasonality** impact on revenue.

> We will look at another factor which is directly related to revenue - **Season** and see how we can optimize our makreting expenditure with it.

> We will try to flag the least performing time of the year so that we can use that expenditure towards most performing months.


####The data doesn't really give convencing results when we do it monthly, let's club it into seasons.

In [43]:
LanguagePerformace = pd.read_sql_query("""WITH channel_rev AS (
    SELECT language,
           ROUND(sum(revenue) ) AS Revenue,
           DENSE_RANK() OVER (ORDER BY SUM(revenue) DESC) AS Rank
      FROM visits AS v,
           subscriptions AS s
     WHERE v.account_id = s.account_id AND 
           region LIKE 'US%'
     GROUP BY 1
)
SELECT language,
       Revenue,
       Rank
  FROM channel_rev;""", con)
LanguagePerformace

,language,Revenue,Rank
0,English,1239973.0,1
1,Spanish,181046.0,2
2,Other,65633.0,3
3,Portuguese,54936.0,4
4,Other_Romance,41673.0,5


In [45]:
print(f"INSIGHT: The data clearly shows that '{LanguagePerformace.iloc[1][0]}' is generating the highest revenue after '{LanguagePerformace.iloc[0][0]}'.")
print('\n')
print(f"RECOMMENDATION: We should add a feature on our landing pages which give user a choice to switch between '{LanguagePerformace.iloc[0][0]}' and '{LanguagePerformace.iloc[1][0]}'")

INSIGHT: The data clearly shows that 'Spanish' is generating the highest revenue after 'English'.


RECOMMENDATION: We should add a feature on our landing pages which give user a choice to switch between 'English' and 'Spanish'


####End of the assesment

---

